In [16]:
import pymc3 as pm
import numpy as np
import scipy.stats as st
import theano.tensor as tt

In [19]:
# with pm.Model() as model1:
#     step_size = pm.HalfNormal('step_size', sd=0.1)
#     nu = pm.HalfNormal('nu', sd=10.)
    
#     sigma = pm.GaussianRandomWalk('sigma', sd=step_size, shape=interval)
#     sigma_weights = tt.repeat(sigma, len(time_series) // interval)
    
#     y_ = pm.StudentT('y', mu=0.0, nu=nu, sd=pm.math.exp(sigma_weights), 
#                      observed=time_series)
    
#     trace = pm.sample(2000, tune=2000)   

In [4]:
pm.GaussianRandomWalk.dist(0.024, shape=8).random(size=100).shape

(100, 8)

In [5]:
pm.GaussianRandomWalk.dist(0.024, shape=(8,3)).random(size=100).shape

(100, 8, 3)

### GaussianRandomWalk variable

In [20]:
bound=pm.distributions.dist_math.bound
alpha, beta = 1.5, 2.
observed = st.weibull_min(alpha, scale=beta).rvs(30)
t = len(observed)
observed

array([0.37728644, 1.08330498, 1.8273372 , 0.96956743, 0.66024372,
       0.54609857, 3.82333264, 0.68288467, 1.13645492, 3.15205414,
       1.36519136, 2.52608288, 3.58613155, 1.49179393, 1.11796344,
       1.02688566, 1.30233172, 2.07894674, 1.12630814, 1.04618896,
       5.00674759, 3.47390823, 3.47591303, 0.65411643, 4.81973732,
       0.92453647, 1.54134617, 0.55154727, 1.9595151 , 0.2597838 ])

In [21]:
def Weibull_logp(alpha, beta, value):
    return bound(tt.log(alpha) - tt.log(beta)
                  + (alpha - 1) * tt.log(value / beta)
                  - (value / beta)**alpha,
                     value >= 0, alpha > 0, beta > 0)

In [22]:
with pm.Model():
    alpha = pm.InverseGamma('alpha',alpha = 1.0,beta = 1.0)
    sd    = pm.InverseGamma('sd',alpha = 2.0,beta = 2.0)
    s     = pm.GaussianRandomWalk('s',sd = sd,shape = t)
    x     = pm.DensityDist('x', Weibull_logp, observed=dict(alpha=alpha, beta=sd, value=observed))
    trace = pm.sample()

<ipython-input-22-2781f331ec2e>:6: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample()
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [s, sd, alpha]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 75 seconds.


MissingInputError: Input 0 of the graph (indices start from 0), used to compute Elemwise{exp,no_inplace}(alpha_log__), was not provided and not given a value. Use the Theano flag exception_verbosity='high', for more information on this error.

In [ ]:
bound=pm.distributions.dist_math.bound
alpha, beta = 1.5, 2.
observed = st.weibull_min(alpha, scale=beta).rvs(30)
t = len(observed)
with pm.Model() as random_walk_model:
    alpha = pm.InverseGamma('alpha', alpha = 1.0, beta = 1.0)
    sd    = pm.InverseGamma('sd', alpha = 2.0, beta = 2.0)
    s     = pm.GaussianRandomWalk('s',sd = sd, shape = t)
    x     = pm.Weibull('x',alpha = alpha, beta = s, observed = observed)
    trace = pm.sample()

In [ ]:
def define_model(n_brands, n_days, poll_brand,
zero_centered_y,
measurement_error_sd):
“”“PyMC3 model for pooling/aggregating voter opinion polls”""

model = pm.Model()
with  model:
    # priors 
    unanchored_house_bias = pm.Cauchy("unanchored_house_bias", 
                                      alpha=0, beta=10, shape=n_brands)
    zero_sum_house_bias = pm.Deterministic('zero_sum_house_bias', 
        var=(unanchored_house_bias - unanchored_house_bias.mean()))
    
    # temporal model
    EARLY_DATA_ITEMS = 5 
    educated_guess = zero_centered_y[:min(EARLY_DATA_ITEMS, len(zero_centered_y))].mean()
    SPAN = 2.0
    starting_distro = pm.Normal('starting_distro', mu=educated_guess, sigma=SPAN) 
    DRIFT = 0.0
    innovation = 0.15 # from experience ... day-to-day change distribution sigma
    grw = pm.GaussianRandomWalk('grw', mu=DRIFT, sigma=innovation, 
                                init=starting_distro, steps=n_days) ### FAILS HERE

    # the observational model
    observed = pm.Normal("observed", 
                         mu=grw[poll_day] 
                            + zero_sum_house_bias[poll_brand.to_list()],
                         sigma=measurement_error_sd, observed=zero_centered_y)
return model